# This post in a glance

Large Language Models (LLMs) have created enormous interest. Here I build and train a Generatively Pretrained Transformer (GPT) following __[Kaparthy's approah](https://www.youtube.com/watch?v=kCc8FmEb1nY)__ in order to understand the under-the-hood components of what makes a GPT work and obtain insights into how GPTs can be applied in real-life commercial applications. <br>

This post is a hands on exploration of the transformer architecture first set out in the seminal 2017 AI paper __[Attention Is All You Need](https://arxiv.org/abs/1706.03762)__, arguably the most important AI paper ever written, and the 2020 GPT-3 paper __[Language Models are Few-Shot Learners](https://arxiv.org/abs/2005.14165)__.

***

#### Insights:

> * DESCRIBE IN ONE SENTENCE WHAT ATTENTION IS AND WHY, THEREFORE, TRANSFORMERS ARE SUCH A SIGNIFICANT STEP IN THE EVOLUTION OF AI <br>
> * DESCRIBE IN ONE SENTENCE WHAT A GPT IS AND WHY IT, TOO, IS AN IMPORTANT STEP ON THE EVOLUTION OF AI <br>
> * SO WHAT? EXPLAIN IN ONE SENTENCE HOW GPTs SHALL TRANSFORM BUSINESS <br>
> * HOW? PROVIDE AN EXAMPLE OF HOW GPTs SHALL TRANSFORM BUSINESS <br>
> * Insights <br>
> * Insights <br>
> * Insights <br>
> * Insights <br>
> * Insights <br>
> * [See Karpathy's conclusions (video 01:54:32) <br>

***

#### The rest of this post ... [DOES WHAT?].

***

# Building and training a GPT

## Baseline (bigram) language model

[Intro]

#### Tokenization

[Details]

#### Train/val split

[Details]

#### Loading the data

[Details]

#### baseline (bigram) language model

[Details]

#### Training the bigram model

[Details]

#### Deployment

[Details]

## Self-attention

#### Averaging past context with for loops (weakest form of aggregation)

[Details]

#### Self-attention: matrix multiply as weighted aggregation

[Details]

#### Using matrix multiply

[Details]

#### Adding softmax

[Details]

#### Positional encoding

[Details]

#### Self-attention

[Details]

Notes

> * Attention as communication <br>

> * Attention has no notion of space, operates over sets

> * There is no communication across batch dimension

> * Encoder blocks vs. decoder blocks

> * Attention vs. self-attention vs. cross-attention

> * "Scaled" self-attention.  Why divide by sqrt(head_size)

## Building the Transformer

#### Inserting a single self-attention block 

[Details]

#### Multi-headed self-attention

[Details]

#### feedforward layers of transformer block

[Details]

#### Residual connections

[Details]

#### Layernorms

[Details]

#### Scaling up

[Details]

## [Notes on Transformer]

#### Encoder vs. decoder vs. both

#### Batched multi-headed self-attention

#### ChatGPT, GPT-3, pretraining vs. finetuning, RLHF

# References

Vaswani, A., et al. (2017).  Attention Is All You Need. arXiv:1706.03762

Brown, T.B., et al. (2020). Language Models are Few-Shot Learners. arXiv:2005.14165

__[Kaparthy's nanoGPT GitHub repo](https://github.com/karpathy/nanoGPT)__

__[Kaparthy's Youtube video](https://www.youtube.com/watch?v=kCc8FmEb1nY)__

__[GitHub repo for Kaparthy's video](https://github.com/karpathy/ng-video-lecture)__

__[Google colab for Kaparthy's video](https://colab.research.google.com/drive/1JMLa53HDuA-i7ZBmqV7ZnA3c_fvtXnx-?usp=sharing)__

***
END